In [1]:
!pip install torch transformers pandas numpy matplotlib scikit-learn nltk gensim tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 75.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 122.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboardx 2.6.2.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.


In [2]:
# reload packages
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
import pandas as pd
import numpy as np
import os
import sys
from semaxis import CoreUtil
from semaxis import SemAxis
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline
import logging

In [3]:
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger(__name__)

In [4]:
# Load the pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

2023-11-10 08:00:16 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2023-11-10 08:00:17 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/vocab.txt HTTP/1.1" 200 0
2023-11-10 08:00:17 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2023-11-10 08:00:17 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/added_tokens.json HTTP/1.1" 404 0
2023-11-10 08:00:17 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2023-11-10 08:00:17 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/special_tokens_map.json HTTP/1.1" 404 0
2023-11-10 08:00:17 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2023-11-10 08:00:17 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/tokenize

In [5]:
# load bert model
from transformers import BertModel

model = BertModel.from_pretrained("../models")

OSError: Can't load config for '../models'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '../models' is the correct path to a directory containing a config.json file

In [ ]:
sa = SemAxis(model, axes_str=CoreUtil.load_wordnet_antonyms_axes())

In [ ]:
sa.compute_document_mean_with_tf(
    ["I like to eat apples and oranges."], min_freq=1
)[0]

In [ ]:
data = []

# get list of file path from data\data\en\dev-articles-subtask-2\*
articles = os.listdir(f"../data/data/en/dev-articles-subtask-2")

for article in articles:
    with open(
        f"../data/data/en/dev-articles-subtask-2/{article}", "r", encoding="utf-8"
    ) as f:
        obj = []

        article_id = article.split(".")[0].replace("article", "")

        obj.append(article_id)

        # read line 3 to n
        lines = f.readlines()[2:]
        obj.append("".join(lines))

        data.append(obj)

# create pandas dataframe
df_articles = pd.DataFrame(data, columns=["article_id", "article"])

# article_id to int
df_articles["article_id"] = df_articles["article_id"].astype(int)

In [ ]:
COLUMNS = [str(c) for c in sorted(sa.axes.keys()) if len(c) == 2]
# Using apply for faster row-wise operations
def process_row(row):
    text = row["article"]
    mean_values = sa.compute_document_mean_with_tf([text], min_freq=1)[0]
    return {col: val for col, val in zip(COLUMNS, mean_values)}

In [ ]:
mean_df = df_articles.apply(process_row, axis=1).apply(pd.Series)

In [ ]:
output_df = pd.concat([df_articles["article_id"], mean_df], axis=1)

In [ ]:
output_df.to_csv("dev_table.tsv", index=False, sep='\t')

# Bootstrapping

Cannot bootstrap as I need a opposite type of data. So e.g. analyze positive and see how the negative looks like. But I have no info about the type of data. So I cannot bootstrap.

> Skip for now

In [30]:
N = 1000
mode = "average"
logger.info(mode)
try:
    os.mkdir(mode)
except:
    pass

09-19 19:39 __main__     INFO     average


In [31]:
def bootstrapping(df, mode, COLUMNS):
    for aspect in df['aspect'].unique():
        for sentiment in ['positive', 'negative']:
            df_r_count = df.query('aspect == @aspect and sentiment == @sentiment').shape[0]
            with open("{}/bootstrap_{}_{}.tsv".format(mode, aspect.replace("/", "-"), sentiment), "w") as fo:
                fo.write("{}\n".format("\t".join(COLUMNS)))
#                 A = df.query('aspect == @aspect').drop(columns=[c for c in df.columns if '(' not in c]).values
                A = df.drop(columns=[c for c in df.columns if '(' not in c]).values

                for i in range(N):
                    if i % 500 == 0:
                        logger.info("{}-{}-{}".format(aspect, sentiment, i))
                    fmean = np.mean(A[np.random.choice(A.shape[0], df_r_count, replace=False), :], axis=0)
                    fo.write("{}\n".format("\t".join([str(v) for v in (fmean)])))

In [32]:
df = pd.read_csv("big_table_by_{}.tsv".format(mode), sep="\t").dropna()
COLUMNS = [c for c in df.columns if '(' in c]
len(COLUMNS)

10

In [33]:
bootstrapping(df, mode, COLUMNS)

KeyError: 'aspect'